In [1]:
### Imports
# General
import numpy as np

# Pymatgen
from pymatgen.electronic_structure.dos import CompleteDos
from pymatgen.electronic_structure.core import OrbitalType, Spin
from pymatgen.io.vasp import Vasprun
from pymatgen import Element

# For integration 
from scipy.integrate import simps

# Filter warning we get from reading in Vasprun.xml files
from warnings import filterwarnings
filterwarnings(action='ignore')

 # Render dicts/lists as json in jupyter notebooks
if get_ipython().__class__.__name__=='ZMQInteractiveShell':
    from IPython.display import JSON
    from json import JSONEncoder, loads
    class MyEncoder(JSONEncoder):
        def default(self, o):
            try:
                return o.as_dict()
            except:
                try:
                    return o.__dict__
                except:
                    return str(o)
    show_json = lambda x : display(JSON(loads(MyEncoder().encode(x))))

## Compare MgO and Cu$_2$O partial DOS
Initially we compare the two extreme cases of MgO, in which the upper valence band is dominated by O p-states, and Cu2O, in which the upper valence band is dominated by Cu d-states.

In [2]:
# Set up
O = Element('O')
Mg = Element('Mg')
Cu = Element('Cu')

s_orb, p_orb, d_orb = OrbitalType(0), OrbitalType(1), OrbitalType(2)

### MgO

In [3]:
# Read in the vasprun and get the whole DOS
MgO = Vasprun('../Data/electronic_structure_data/1_MgO/vasprun.xml')
MgO_DOS = MgO.complete_dos

# Isolate O p and Mg s orbital contributions 
O_DOS = MgO_DOS.get_element_spd_dos(el=O)
O_p_DOS = O_DOS[p_orb]

Mg_DOS = MgO_DOS.get_element_spd_dos(el=Mg)
Mg_s_DOS = Mg_DOS[s_orb]

# Get the energies and densities from up and down spin channel 
energies = O_p_DOS.energies
O_p_tot = O_p_DOS.densities[Spin(1)] + O_p_DOS.densities[Spin(-1)]
Mg_s_tot = Mg_s_DOS.densities[Spin(1)] + Mg_s_DOS.densities[Spin(-1)]

# Integrate over some portion of the upper valence band 
efermi = MgO_DOS.efermi
int_limit = 1.0 # eV integration limit

upper_vbm_O_p = np.array([[en, dens] for en, dens in zip(energies, O_p_tot) 
                         if efermi - int_limit <= en <= efermi])
upper_vbm_O_p = upper_vbm_O_p.transpose()

upper_vbm_Mg_s = np.array([[en, dens] for en, dens in zip(energies, Mg_s_tot) 
                         if efermi - int_limit <= en <= efermi])
upper_vbm_Mg_s = upper_vbm_Mg_s.transpose()

O_p_int = simps(upper_vbm_O_p[1], upper_vbm_O_p[0])
Mg_s_int = simps(upper_vbm_Mg_s[1], upper_vbm_Mg_s[0])

# Calculate the percentage of each contribution to VBM 
vbm_int = O_p_int + Mg_s_int
O_p_contrib = O_p_int/vbm_int*100
Mg_s_contrib = Mg_s_int/vbm_int*100
print(f"O p contribution: {O_p_contrib :.2f}%")
print(f"Mg s contribution: {Mg_s_contrib :.2f}%")   

O p contribution: 99.93%
Mg s contribution: 0.07%


### Cu$_2$O
We now do the same for Cu2O, this time considering the Cu d-states.

In [4]:
# Read in the vasprun and get the whole DOS
Cu2O = Vasprun('../Data/electronic_structure_data/2_Cu2O/vasprun.xml')
Cu2O_DOS = Cu2O.complete_dos

# Isolate O p and Cu d orbital contributions 
O_DOS = Cu2O_DOS.get_element_spd_dos(el=O)
O_p_DOS = O_DOS[p_orb]

Cu_DOS = Cu2O_DOS.get_element_spd_dos(el=Cu)
Cu_d_DOS = Cu_DOS[d_orb]

# Get the energies and densities from up and down spin channel 
energies = O_p_DOS.energies
O_p_tot = O_p_DOS.densities[Spin(1)] + O_p_DOS.densities[Spin(-1)]
Cu_d_tot = Cu_d_DOS.densities[Spin(1)] + Cu_d_DOS.densities[Spin(-1)]

# Integrate over some portion of the upper valence band 
efermi = Cu2O_DOS.efermi
int_limit = 1.0 # eV integration limit

upper_vbm_O_p = np.array([[en, dens] for en, dens in zip(energies, O_p_tot) 
                         if efermi - int_limit <= en <= efermi])
upper_vbm_O_p = upper_vbm_O_p.transpose()

upper_vbm_Cu_d = np.array([[en, dens] for en, dens in zip(energies, Cu_d_tot) 
                         if efermi - int_limit <= en <= efermi])
upper_vbm_Cu_d = upper_vbm_Cu_d.transpose()

O_p_int = simps(upper_vbm_O_p[1], upper_vbm_O_p[0])
Cu_d_int = simps(upper_vbm_Cu_d[1], upper_vbm_Cu_d[0])

# Calculate the percentage of each contribution to VBM 
vbm_int = O_p_int + Cu_d_int
O_p_contrib = O_p_int/vbm_int*100
Cu_d_contrib = Cu_d_int/vbm_int*100
print(f"O p contribution: {O_p_contrib :.2f}%")
print(f"Cu d contribution: {Cu_d_contrib :.2f}%") 

O p contribution: 13.43%
Cu d contribution: 86.57%


## General function
We now use a general function to compare M d-orbital to O p-orbital contribution to the upper VB in a Li-M-O battery cathode. As before, we only consider the M-d and O-p contributions and report each as a percentage of the sum of these. As a demonstration, we look at the different contributions to the upper VB in LiRuO2, LiRu2O3, LiWO2 and Li2WO3. 

In [9]:
def upper_VB_contribs(vasprun, int_limit=1.0):
    ''' Calculate the percentage contributions of M d-orbitals and 
    O p-orbitals to the upper valence band of Li-M-O systems. 
    Args:
        vasprun: Pymatgen vasprun file. 
        int_limit (float): Limit to integrate over; how far down the 
        valence band to include as 'upper valence band'.
    '''
    tm = [m for m in vasprun.final_structure.composition 
          if m.symbol not in ['Li','O']][0]
    DOS = vasprun.complete_dos
    
    # Isolate O p- and M d-orbital contributions 
    O_DOS = DOS.get_element_spd_dos(el=O)
    O_p_DOS = O_DOS[p_orb]

    M_DOS = DOS.get_element_spd_dos(el=tm)
    M_d_DOS = M_DOS[d_orb]

    # Get the energies and densities from up and down spin channel 
    energies = O_p_DOS.energies
    O_p_tot = O_p_DOS.densities[Spin(1)] + O_p_DOS.densities[Spin(-1)]
    M_d_tot = M_d_DOS.densities[Spin(1)] + M_d_DOS.densities[Spin(-1)]

    # Integrate over some portion of the upper valence band 
    efermi = DOS.efermi
    int_limit = 1.0 # eV integration limit

    upper_vbm_O_p = np.array([[en, dens] for en, dens in zip(energies, O_p_tot) 
                             if efermi - int_limit <= en <= efermi])
    upper_vbm_O_p = upper_vbm_O_p.transpose()

    upper_vbm_M_d = np.array([[en, dens] for en, dens in zip(energies, M_d_tot) 
                             if efermi - int_limit <= en <= efermi])
    upper_vbm_M_d = upper_vbm_M_d.transpose()

    O_p_int = simps(upper_vbm_O_p[1], upper_vbm_O_p[0])
    M_d_int = simps(upper_vbm_M_d[1], upper_vbm_M_d[0])

    # Calculate the percentage of each contribution to VBM 
    vbm_int = O_p_int + M_d_int
    O_p_contrib = O_p_int/vbm_int*100
    M_d_contrib = M_d_int/vbm_int*100
    print(vasprun.final_structure.composition.reduced_formula)
    print(f"O p contribution: {O_p_contrib :.2f}%")
    print(f"{tm.symbol} d contribution: {M_d_contrib :.2f}%") 

In [13]:
LiRuO2 = Vasprun('../Data/electronic_structure_data/3_LiRuO2/vasprun.xml')
upper_VB_contribs(LiRuO2)
Li2RuO3 = Vasprun('../Data/electronic_structure_data/4_Li2RuO3/vasprun.xml')
upper_VB_contribs(Li2RuO3)

LiWO2 = Vasprun('../Data/electronic_structure_data/5_LiWO2/vasprun.xml')
upper_VB_contribs(LiWO2)
Li2WO3 = Vasprun('../Data/electronic_structure_data/6_Li2WO3/vasprun.xml')
upper_VB_contribs(Li2WO3)

LiRuO2
O p contribution: 20.35%
Ru d contribution: 79.65%
Li2RuO3
O p contribution: 34.65%
Ru d contribution: 65.35%
LiWO2
O p contribution: 18.12%
W d contribution: 81.88%
Li2WO3
O p contribution: 24.85%
W d contribution: 75.15%
